In [1]:
# Importing Libraries

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet import preprocess_input
from imutils.object_detection import non_max_suppression
import imutils
import time

In [2]:
def sliding_window(image, step, ws):
    # slide a window across the image
    for y in range(0, image.shape[0] - ws[1], step):
        for x in range(0, image.shape[1] - ws[0], step):
            # yield the current window
            yield (x, y, image[y:y + ws[1], x: x + ws[0]])

def image_pyramid(image, scale=1.5, min_size=(224, 224)):
    # yield the original image
    yield image

    # keep looping over the image pyramid
    while True:
        # compute the dimensions of the next image in the pyramid
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)

        # if the resized image does not meet the supplied minimum size, then stop constructing the pyramid
        if image.shape[0] < min_size[1] or image.shape[1] < min_size[0]:
            break

        # yield the next image in the pyramid
        yield image

In [11]:
# Load your model
model_path = 'C:/Users/Lenovo/Videos/Project/RECIPE GENERATOR/Jupyter/mobileNet.h5'
vegetable_model = load_model(model_path)

# Your image path
img_path = 'C:/Users/Lenovo/Videos/Project/RECIPE GENERATOR/Jupyter/test_image.jpg'
orig = cv2.imread(img_path)

In [12]:
# Class Names
root_path = 'C:/Users/Lenovo/Videos/Project/RECIPE GENERATOR/Jupyter/split_refined_data/train/'
class_names = sorted(os.listdir(root_path))
n_classes = len(class_names)

# Class Distribution
class_dis = [len(os.listdir(root_path + name)) for name in class_names]


# Show
print(f"Total Number of Classes : {n_classes} /nClass Names : {class_names}")

Total Number of Classes : 26 /nClass Names : ['BEAN', 'BEETROOT', 'BITTER GOURD', 'BOTTLE GOURD', 'CABBAGE', 'CAPSICUM', 'CARROT', 'CAULIFLOWER', 'CHILLI', 'CORN', 'CUCUMBER', 'EGG', 'EGGPLANT', 'GARLIC', 'GINGER', 'LEMON', 'LETTUCE', 'OKRA', 'ONION', 'PEAS', 'POTATO', 'PUMPKIN', 'RADDISH', 'SPINACH', 'TOMATO', 'TURNIP']


In [32]:
import random
# Set parameters
MIN_CONFIDENCE = 0.95  # Adjust this threshold as needed
WIDTH = 900
PYR_SCALE = 1.5
WIN_STEP = 8
INPUT_SIZE = (224, 224)

# initialize the image pyramid
pyramid = image_pyramid(orig, scale=PYR_SCALE, min_size=INPUT_SIZE)

# initialize two lists, one to hold the ROIs generated from the image pyramid and sliding window,
# and another list used to store the (x,y)-coordinate of where the ROI was in the original image
rois = []
locs = []

# loop over the image pyramid
for image in pyramid:
    # for each layer of the image pyramid, loop over the sliding window locations
    for (x, y, roi_orig) in sliding_window(image, WIN_STEP, INPUT_SIZE):
        # randomly change the size of the bounding box within a specified range
        roi_size = (
            random.randint(50, 200),  # random width between 100 and 200 pixels
            random.randint(50, 200),   # random height between 75 and 150 pixels
        )

        # take the ROI and pre-process it so we can later classify the region using Keras/Tensorflow
        roi = cv2.resize(roi_orig, (224, 224))  # Resize to a fixed size
        roi = img_to_array(roi)
        roi = preprocess_input(roi)

        # update our list of ROIs and associated coordinates
        rois.append(roi)
        locs.append((x, y, x + roi_size[0], y + roi_size[1]))

# convert and ROIs to a Numpy array
rois = np.array(rois, dtype='float32')

# classify each of the proposal ROIs using your model
predictions = vegetable_model.predict(rois)

# initialize the list to store detected vegetables
detected_vegetables = []

# initialize a set to keep track of unique detected vegetable names
unique_vegetable_names = set()

# Loop over the predictions
for i, pred in enumerate(predictions):
    # Get the predicted class index
    predicted_class_index = np.argmax(pred)

    # Get the class name
    predicted_class = class_names[predicted_class_index]

    # Get the confidence/probability score
    confidence = pred[predicted_class_index]

    # Filter out weak detections by ensuring the predicted probability is greater than the minimum probability
    if confidence >= MIN_CONFIDENCE:
        # Check if the vegetable name is not already present in the set
        if predicted_class not in unique_vegetable_names:
            # Add the detected vegetable to the list
            detected_vegetables.append({
                'class': predicted_class,
                'confidence': confidence,
                'box': locs[i]
            })
            
            # Add the vegetable name to the set to avoid repetition
            unique_vegetable_names.add(predicted_class)

# Create a list of detected vegetable names
detected_vegetable_names = [veg['class'] for veg in detected_vegetables]

# Print the list of detected vegetable names
print("Detected Vegetable Names:", detected_vegetable_names)

# # Print the list of detected vegetables
# for veg in detected_vegetables:
#     print(f"Class: {veg['class']}, Confidence: {veg['confidence']}, Bounding Box: {veg['box']}")

# Draw bounding boxes on the original image for visualization
clone = orig.copy()
for veg in detected_vegetables:
    (startX, startY, endX, endY) = veg['box']
    cv2.rectangle(clone, (startX, startY), (endX, endY), (0, 255, 0), 2)

# # Show the output after
# cv2.imshow("Detected Vegetables", clone)
# cv2.waitKey(0)
# Save the output image after drawing bounding boxes
output_path = 'C:/Users/Lenovo/Videos/Project/RECIPE GENERATOR/Jupyter/detected_objects.jpg'
cv2.imwrite(output_path, clone)
print(f"Output image saved at: {output_path}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 261ms/step
Detected Vegetable Names: ['CUCUMBER', 'EGGPLANT']
Output image saved at: C:/Users/Lenovo/Videos/Project/RECIPE GENERATOR/Jupyter/detected_objects.jpg
